In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyspark
from functools import reduce
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import desc,asc, monotonically_increasing_id, split, size, length

In [32]:
def get_start_day(start_day):
  import datetime
  date_arr = start_day.split("/")
  end_date =  datetime.datetime(int('20' +date_arr[2]), int(date_arr[0]), int(date_arr[1]))    
  start_date = end_date + datetime.timedelta(days=-20)
  return start_date.strftime("%Y%m%d")
  

In [33]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType
get_start_day_udf = udf(get_start_day)

In [34]:
def get_end_day(start_day):
  import datetime
  date_arr = start_day.split("/")
  end_date =  datetime.datetime(int('20' +date_arr[2]), int(date_arr[0]), int(date_arr[1]))    
  start_date = end_date + datetime.timedelta(days=-20)
  return end_date.strftime("%Y%m%d")


from pyspark.sql.functions import udf
from pyspark.sql.types import StructType
get_end_day_udf = udf(get_end_day)
  

In [35]:
from statistics import mean
def get_average( array):
  return mean(array)


from pyspark.sql.functions import udf
from pyspark.sql.types import StructType
get_average_udf = udf(get_average)

In [36]:
def variance_plot(col, df):
  col_series = df.groupBy(col).count().sort(desc("count"))
  total_sum = col_series.agg({"count":"sum"}).collect()[0][0]
  col_series = col_series.withColumn('variances', udf(lambda x: int(x)/total_sum)('count'))
  windowval = (Window.orderBy(F.col('count').desc()).rangeBetween(Window.unboundedPreceding, 0))
  col_series = col_series.withColumn('cum_sum', F.sum('variances').over(windowval))
  plt.plot(col_series.select('cum_sum').collect())
  display(plt.show())

In [37]:
def missing_values_table(df):
   
    # Total missing values
    mis_val = df.isnull().sum()
    
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    
    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    
    # Sort the table by percentage of missing descending
    # .iloc[:, 1]!= 0: filter on missing missing values not equal to zero
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(2)  # round(2), keep 2 digits
    
    # Print some summary information
    print("Your slelected dataframe has {} columns.".format(df.shape[1]) + '\n' + 
    "There are {} columns that have missing values.".format(mis_val_table_ren_columns.shape[0]))
    
    # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [38]:
def variance_plot(col, df):
  col_series = df.groupBy(col).count().sort(desc("count"))
  total_sum = col_series.agg({"count":"sum"}).collect()[0][0]
  col_series = col_series.withColumn('variances', udf(lambda x: int(x)/total_sum)('count'))
  windowval = (Window.orderBy(F.col('count').desc()).rangeBetween(Window.unboundedPreceding, 0))
  col_series = col_series.withColumn('cum_sum', F.sum('variances').over(windowval))
  plt.plot(col_series.select('cum_sum').collect())
  display(plt.show())

In [39]:
def get_n_values(n, col, df):
  col_series = df.groupBy(col).count().sort(desc("count"))
  vals = col_series.filter(~F.col(col).isin('nan','')).head(n) ##number 20 should be n
  final_values = [x[0] for x in vals]
  return final_values

In [40]:
def varianceplot_nvalue(col, df):
  col_series = df.groupBy(col).count().sort(desc("count"))
  total_sum = col_series.agg({"count":"sum"}).collect()[0][0]
  col_series = col_series.withColumn('variances', udf(lambda x: int(x)/total_sum)('count'))
  windowval = (Window.orderBy(F.col('count').desc()).rangeBetween(Window.unboundedPreceding, 0))
  col_series = col_series.withColumn('cum_sum', F.sum('variances').over(windowval))
  n = col_series.filter(F.col('cum_sum') <= 0.98).count() 
  plt.plot(col_series.select('cum_sum').collect())
  return plt.show(), n

In [41]:
def new_col_values(val):
    result = val if val in final_values else ''
    return result